# Pandas II

## Indexação Hierárquica

In [61]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(42)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))

In [5]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.374540
   2    0.950714
   3    0.731994
b  1    0.598658
   3    0.156019
c  1    0.155995
   2    0.058084
d  2    0.866176
   3    0.601115
dtype: float64

In [6]:
data["b"]

1    0.598658
3    0.156019
dtype: float64

In [7]:
data["b":"c"]

b  1    0.598658
   3    0.156019
c  1    0.155995
   2    0.058084
dtype: float64

In [8]:
data.loc[["b", "d"]]

b  1    0.598658
   3    0.156019
d  2    0.866176
   3    0.601115
dtype: float64

In [9]:
data.loc[:, 2] # todos os valores que possuem 2 no segundo índice

a    0.950714
c    0.058084
d    0.866176
dtype: float64

In [10]:
data.unstack()

,1,2,3
a,0.374540,0.950714,0.731994
b,0.598658,NaN,0.156019
c,0.155995,0.058084,NaN
d,NaN,0.866176,0.601115


In [11]:
data.unstack().stack()

a  1    0.374540
   2    0.950714
   3    0.731994
b  1    0.598658
   3    0.156019
c  1    0.155995
   2    0.058084
d  2    0.866176
   3    0.601115
dtype: float64

In [12]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [14]:
# É possível ver quantos níveis um índice tem com o atributo nlevels
frame.index.nlevels

2

In [15]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [16]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [17]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [18]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Reshape e Pivoting

In [19]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [20]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [21]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [22]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [23]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [24]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [25]:
data2.unstack().stack(dropna=False)

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [26]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [27]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [28]:
df.unstack(level="state").stack(level="side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [161]:
data = pd.read_csv("bases/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [162]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [163]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [160]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

TypeError: reindex() got an unexpected keyword argument "columns"

In [159]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

AttributeError: 'Series' object has no attribute 'stack'

In [42]:
long_data[:5]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340


In [43]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


pivot é equivalente a criar um índice hierárquico usando `set_index()`seguido da chamada do método `stack()`. 

In [44]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                
item        infl   realgdp unemp
date                            
1959-01-01  0.00  2710.349   5.8
1959-04-01  2.34  2778.801   5.1
1959-07-01  2.74  2775.488   5.3
1959-10-01  0.27  2785.204   5.6
1960-01-01  2.31  2847.699   5.2

In [45]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [46]:
melted = pd.melt(df, id_vars="key")
melted
#a coluna key pode ser um indicador de grupo. Quando usamos pd.melt(), precisamos indicar
#quais colunas (caso haja) sao indicadores de grupo

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [47]:
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [48]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [ ]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

In [ ]:
pd.melt(df, value_vars=["A", "B", "C"])

In [ ]:
df = pd.DataFrame.from_dict({"Nome": ['Loja1', 'Loja2', 'Loja3', 'Loja4'],
                             "10/05/2023": np.random.randint(10,200, size=(1,4))[0],
                             "10/06/2023": np.random.randint(12,200, size=(1,4))[0],
                             "10/07/2023": np.random.randint(12,200, size=(1,4))[0],
                             "10/08/2023": np.random.randint(12,200, size=(1,4))[0]}, orient='columns')
df

In [ ]:
df.melt(id_vars='Nome', var_name='Data', value_name='Vendas_Total')

## Agregação de dados e operações Group

In [78]:
np.random.seed(42)
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
                                      dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,0.496714,0.767435
1,a,2,-0.138264,-0.469474
2,None,1,0.647689,0.542560
3,b,2,1.523030,-0.463418
4,b,1,-0.234153,-0.465730
5,a,<NA>,-0.234137,0.241962
6,None,1,1.579213,-1.913280


In [80]:
grouped = df["data1"].groupby(df["key1"])
grouped.mean()

key1
a    0.041438
b    0.644438
Name: data1, dtype: float64

In [81]:
grouped

In [82]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1       0.496714
      2      -0.138264
b     1      -0.234153
      2       1.523030
Name: data1, dtype: float64

In [83]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [85]:
df.groupby("key1", dropna=False).size()
#para computar o número de valores nao-nulos, use o método count()

key1
a      3
b      2
NaN    2
dtype: int64

In [86]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  0.496714  0.767435
1    a     2 -0.138264 -0.469474
5    a  <NA> -0.234137  0.241962
b
  key1  key2     data1     data2
3    b     2  1.523030 -0.463418
4    b     1 -0.234153 -0.465730


In [87]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1  0.496714  0.767435
('a', 2)
  key1  key2     data1     data2
1    a     2 -0.138264 -0.469474
('b', 1)
  key1  key2     data1    data2
4    b     1 -0.234153 -0.46573
('b', 2)
  key1  key2    data1     data2
3    b     2  1.52303 -0.463418


In [88]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces['b']

,key1,key2,data1,data2
3,b,2,1.523030,-0.463418
4,b,1,-0.234153,-0.465730


In [89]:
df.groupby("key1")["data1"]

In [90]:
df["data1"].groupby(df["key1"])

In [91]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.767435
     2    -0.469474
b    1    -0.465730
     2    -0.463418

In [92]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-1.724918,-0.562288,-1.012831,0.314247,-0.908024
Steve,-1.412304,1.465649,-0.225776,0.067528,-1.424748
Wanda,-0.544383,NaN,NaN,0.375698,-0.600639
Jill,-0.291694,-0.601707,1.852278,-0.013497,-1.057711
Trey,0.822545,-1.220844,0.208864,-1.959670,-1.328186


In [93]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}
#incluí a chave f para pontuar que grupos nao usados nao geram problemas

In [94]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

,blue,red
Joe,-0.698584,-3.195229
Steve,-0.158248,-1.371403
Wanda,0.375698,-1.145021
Jill,1.838781,-1.951111
Trey,-1.750807,-1.726485


In [95]:
df

,key1,key2,data1,data2
0,a,1,0.496714,0.767435
1,a,2,-0.138264,-0.469474
2,None,1,0.647689,0.542560
3,b,2,1.523030,-0.463418
4,b,1,-0.234153,-0.465730
5,a,<NA>,-0.234137,0.241962
6,None,1,1.579213,-1.913280


In [96]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     5   -0.234137
      1   -0.138264
b     4   -0.234153
      3    1.523030
Name: data1, dtype: float64

In [97]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,0.730851,1.236909
b,1,1.757183,0.002312


In [100]:
tips = pd.read_csv("bases/tips.csv")
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [101]:
grouped = tips.groupby(["day", "smoker"])

In [102]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [103]:
grouped_pct.agg(["mean", "std", peak_to_peak]) 

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [104]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples) 

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [105]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [106]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [107]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [108]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")
#group_keys = False fará com que o índice hierárquico seja suprimido

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [109]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,0.196861,1.047098
1,0.738467,1.683928
2,0.171368,-0.458884
3,-0.115648,1.078681
4,-0.301104,-0.038508


In [110]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(8)

0     (-1.468, 0.306]
1      (0.306, 2.079]
2     (-1.468, 0.306]
3     (-1.468, 0.306]
4     (-1.468, 0.306]
5    (-3.248, -1.468]
6     (-1.468, 0.306]
7     (-1.468, 0.306]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.248, -1.468] < (-1.468, 0.306] < (0.306, 2.079] < (2.079, 3.853]]

In [111]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.248, -1.468] data1 -3.241267 -1.478522     58 -1.853109
                 data2 -1.768439  2.157308     58  0.111824
(-1.468, 0.306]  data1 -1.464375  0.301547    556 -0.435644
                 data2 -2.940389  3.193108    556  0.075193
(0.306, 2.079]   data1  0.307300  2.075401    364  0.919920
                 data2 -3.019512  3.137749    364  0.032265
(2.079, 3.853]   data1  2.092387  3.852731     22  2.432890
                 data2 -1.993736  1.749577     22 -0.178836

In [ ]:
tips.head()

In [ ]:
tips.pivot_table(index=["day", "smoker"])

In [ ]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

In [ ]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len)

In [ ]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, fill_value=0)

In [112]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")
data.head()

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed


In [113]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)
#margins = True representa a soma (nesse caso) dos elementos em cada eixo

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [ ]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

## TODO Section

1. Usando o dataset Pokemon.csv, agrupe os pokemons por Type 1 e retorne uma Série ordenada pela quantidade em ordem decrescente

In [114]:
poke = pd.read_csv('bases/Pokemon.csv')
poke.head(n=10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


In [126]:
#resposta 1
grouped = poke.groupby('Type 1')
grouped['Name'].count().sort_values(ascending=0)

Type 1
Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Ghost        32
Ground       32
Dragon       32
Dark         31
Poison       28
Fighting     27
Steel        27
Ice          24
Fairy        17
Flying        4
Name: Name, dtype: int64

2. Considerando o dataset tips, agrupe os dados por smoker e, para a coluna tip_pct, retorne a aplicação do método `describe()`. 

In [235]:
#resposta 2
tips = pd.read_csv('bases/tips.csv')
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head(n=10)

grouped = tips.groupby('smoker')
grouped['tip_pct'].agg('describe')

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [236]:
tips.groupby('smoker')['tip_pct'].describe()

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


3. Considerando a Series abaixo, crie uma função que preencha os valores faltantes com a média de cada grupo específico.

In [167]:
np.random.seed(42)
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data[["Vermont", "Nevada", "Idaho"]] = np.nan

In [233]:
# Resposta 3
df = pd.DataFrame(data.values, index = data.index, columns=['value'])
df['Region'] = group_key
grouped = df.groupby('Region')
dict = grouped.mean().to_dict()['value']
map = df['Region'].map(dict)

df['value'].fillna(map, inplace=True)
print(my_dict)
print(df)

{'East': 0.6271599027493578, 'West': 0.6725297203920277}
               value Region
Ohio        0.496714   East
New York   -0.138264   East
Vermont     0.627160   East
Florida     1.523030   East
Oregon     -0.234153   West
Nevada      0.672530   West
California  1.579213   West
Idaho       0.672530   West


In [234]:
# Resposta 3 do professor
def fill_mean(group):
    return group.fillna(group.mean());

data.groupby(group_key).apply(fill_mean)

Ohio          0.496714
New York     -0.138264
Vermont       0.627160
Florida       1.523030
Oregon       -0.234153
Nevada        0.672530
California    1.579213
Idaho         0.672530
dtype: float64